In [168]:
import keras
from keras import optimizers
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras import backend as K
K.set_image_dim_ordering('th')
from sklearn.model_selection import train_test_split
from keras.layers.core import Activation

In [2]:
seed = 7
np.random.seed(seed)
    
#read both data sets
train_data = pd.read_csv("data/train.csv")
test_data = pd.read_csv("data/test.csv")

In [69]:
train_info = train_data[['Prediction','Id',"NextId","Position"]]
test_info = test_data[['Id',"NextId","Position"]]

In [5]:
X_train = train_data.drop(['Prediction','Id',"NextId","Position"],axis=1).as_matrix()
X_test = test_data.drop(['Prediction','Id',"NextId","Position"],axis=1).as_matrix()

In [110]:
unique_words=set()
str=""
for index, row in train_data.iterrows():
    str += row["Prediction"]
    if row["NextId"] == -1:
        unique_words.add(str)
        str = ""

In [122]:
unique_words_list = list(unique_words)
unique_words_list = sorted(unique_words_list, key=len)

In [137]:
words_map = {}
for word in unique_words_list:
    words_map[word] = keras.utils.to_categorical(len(words_map) +1, 56).reshape(1,56)

In [149]:
def create_words(info, data, train):
    words = np.empty((0, 16,112), int)
    predictions = np.empty((0, 56), int)
    word_length = np.empty((0, 1), int)
    
    for index, row in info.iterrows():
        if(row["Position"] == 1):
            word = data[index].reshape(1,16,8)
            if(train):
                str = row["Prediction"]
        elif(row["NextId"] == -1):
            word_length = np.append(word_length, row["Position"])
            word = np.append(word, data[index].reshape(1,16,8), axis=2)
            if(train):
                str += row["Prediction"]
                
            length = word[0][0].size
            while (length < 112):
                word = np.append(word, np.full((1,16, 8), 0), axis=2)
                length = word[0][0].size
                
            words =np.append(words, word, axis=0)
            if(train):
                predictions =np.append(predictions, words_map[str], axis=0)
        else:
            word = np.append(word, data[index].reshape(1,16,8), axis=2)
            if(train):
                str += row["Prediction"]
    
    return words, word_length, predictions    

In [77]:
test_words, test_word_length = create_words(test_info, X_test, False)

In [ ]:
train_words, train_word_length, predictions = create_words(train_info, X_train, True)

In [150]:
print(train_words.shape, predictions.shape, train_word_length.shape)

(5487, 16, 112) (5487, 56) (5487,)


In [151]:
print(test_words.shape, test_word_length.shape)

(1390, 16, 112) (1390,)


In [169]:
def baseline_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(1, 16, 112), activation='relu', padding='same'))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(224, activation='relu'))
    model.add(Dense(56, activation='softmax'))
    sgd = optimizers.SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model

In [172]:
model = baseline_model()
model.fit(train_words.reshape(5487,1, 16, 112) , predictions, epochs=10, batch_size=20, verbose=1)

Epoch 1/10
5487/5487 [==============================] - 151s 28ms/step - loss: 3.1989 - acc: 0.1637
Epoch 2/10
5487/5487 [==============================] - 151s 28ms/step - loss: 0.6688 - acc: 0.7862
Epoch 3/10
5487/5487 [==============================] - 150s 27ms/step - loss: 0.2973 - acc: 0.9050
Epoch 4/10
5487/5487 [==============================] - 153s 28ms/step - loss: 0.2077 - acc: 0.9322
Epoch 5/10
5487/5487 [==============================] - 153s 28ms/step - loss: 0.1371 - acc: 0.9526
Epoch 6/10
5487/5487 [==============================] - 154s 28ms/step - loss: 0.1161 - acc: 0.9608
Epoch 7/10
5487/5487 [==============================] - 154s 28ms/step - loss: 0.0949 - acc: 0.9690
Epoch 8/10
5487/5487 [==============================] - 154s 28ms/step - loss: 0.0825 - acc: 0.9723
Epoch 9/10
5487/5487 [==============================] - 155s 28ms/step - loss: 0.0762 - acc: 0.9739
Epoch 10/10
5487/5487 [==============================] - 154s 28ms/step - loss: 0.0664 - acc: 0.9781

In [203]:
model.fit(train_words.reshape(5487,1, 16, 112) , predictions, epochs=2, batch_size=20, verbose=1)

Epoch 1/2
5487/5487 [==============================] - 150s 27ms/step - loss: 0.0375 - acc: 0.9863
Epoch 2/2
5487/5487 [==============================] - 149s 27ms/step - loss: 0.0378 - acc: 0.9885


In [204]:
predict = model.predict(test_words.reshape(1390,1, 16, 112))

In [205]:
classes = [unique_words_list[np.argmax(p)-1] for p in predict]

In [206]:
for i in range(0, len(classes)):
    if(len(classes[i]) != test_word_length[i]):
        print("Not matching", classes[i], test_word_length[i], i)

In [207]:
results = np.empty((0, 1))
for i in range(0, len(classes)):
    results = np.append(results, list(classes[i]))

In [208]:
results.size

10584

In [209]:
np.savetxt("final_results.csv", np.dstack((np.arange(0, results.size),results))[0],"%s,%s",header="Id,Prediction")

In [210]:
from IPython.display import FileLink, FileLinks
csv_fn='final_results.csv'
FileLink(csv_fn)

/home/user/Projects/PredictionComp1/final_results.csv